In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import date
from sklearn.metrics import r2_score
    
symbol="^NSEBANK"
start_date="2020-01-01"
end_date=date.today()
interval="1d"
banknifty=yf.download(symbol,start=start_date,end=end_date,interval=interval)
print(banknifty)
banknifty.to_csv('banknifty.csv')



# Load historical Bank Nifty data from CSV file
data = pd.read_csv('banknifty.csv')
banknifty_data = data['Close'].values

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
banknifty_data = scaler.fit_transform(banknifty_data.reshape(-1, 1))

# Split the data into training and testing sets
train_data = banknifty_data[:800]
test_data = banknifty_data[800:]

# Prepare the data for LSTM training
def create_dataset(data, lookback=1):
    X, Y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:(i+lookback), 0])
        Y.append(data[i+lookback, 0])
    return np.array(X), np.array(Y)

lookback = 10  # Number of previous time steps to consider
train_X, train_Y = create_dataset(train_data, lookback)
test_X, test_Y = create_dataset(test_data, lookback)

# Reshape the input data to match LSTM requirements (samples, timesteps, features)
train_X = np.reshape(train_X, (train_X.shape[0], lookback, 1))
test_X = np.reshape(test_X, (test_X.shape[0], lookback, 1))

# Build the LSTM model
model = Sequential()
model.add(LSTM(34, input_shape=(lookback, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(train_X, train_Y, epochs=10, batch_size=1, verbose=2)

# Predict Bank Nifty on train and test data
train_predictions = model.predict(train_X)
test_predictions = model.predict(test_X)
train_predictions = scaler.inverse_transform(train_predictions)
test_predictions = scaler.inverse_transform(test_predictions)

# Prepare plot data
train_actual = scaler.inverse_transform(train_Y.reshape(-1, 1))
test_actual = scaler.inverse_transform(test_Y.reshape(-1, 1))

# Plot the predicted and actual Bank Nifty values
plt.figure(figsize=(20, 6))
plt.plot(train_actual, label='Actual (Train)')
plt.plot(train_predictions, label='Predicted (Train)')
plt.plot(len(train_actual) + np.arange(len(test_actual)), test_actual, label='Actual (Test)')
plt.plot(len(train_actual) + np.arange(len(test_predictions)), test_predictions, label='Predicted (Test)')
plt.xlabel('Time')
plt.ylabel('Bank Nifty Value')
plt.title('Bank Nifty - Actual vs. Predicted')
plt.legend()
plt.show()


# Calculate MSE and RMSE for test predictions
r2 = r2_score(test_actual, test_predictions)

print("R Squared Score:", r2)
#print("Root Mean Squared Error (RMSE):", rmse)
